In [ ]:
import boto3
from botocore.exceptions import ClientError

def validate_iam_role(role_name):
    iam = boto3.client('iam')
    try:
        role = iam.get_role(RoleName=role_name)
        policies = iam.list_attached_role_policies(RoleName=role_name)
        inline_policies = iam.list_role_policies(RoleName=role_name)
        return role, policies, inline_policies
    except ClientError as e:
        print(f"Error retrieving role: {e}")
        return None

def check_permissions(service_client, action, **kwargs):
    try:
        method = getattr(service_client, action)
        response = method(**kwargs)
        print(f"{action} succeeded: {response}")
        return True
    except ClientError as e:
        print(f"{action} failed: {e}")
        return False

def test_sagemaker_permissions():
    sagemaker_client = boto3.client('sagemaker')
    print("Testing SageMaker ListTrainingJobs permission")
    check_permissions(sagemaker_client, 'list_training_jobs')
    
    print("Testing SageMaker DescribeTrainingJob permission")
    check_permissions(sagemaker_client, 'describe_training_job', TrainingJobName='NonExistentJob')
    
    print("Testing SageMaker InvokeEndpoint permission")
    check_permissions(sagemaker_client, 'invoke_endpoint', EndpointName='NonExistentEndpoint', Body=b'')

def test_athena_permissions():
    athena_client = boto3.client('athena')
    print("Testing Athena ListQueryExecutions permission")
    check_permissions(athena_client, 'list_query_executions')
    
    print("Testing Athena StartQueryExecution permission")
    check_permissions(athena_client, 'start_query_execution', QueryString='SELECT 1', ResultConfiguration={'OutputLocation': 's3://nonexistent-bucket/'})

def test_quicksight_permissions():
    quicksight_client = boto3.client('quicksight')
    print("Testing QuickSight ListDashboards permission")
    check_permissions(quicksight_client, 'list_dashboards', AwsAccountId='NonExistentAccountId')

    print("Testing QuickSight DescribeDashboard permission")
    check_permissions(quicksight_client, 'describe_dashboard', AwsAccountId='NonExistentAccountId', DashboardId='NonExistentDashboardId')

def test_s3_permissions():
    s3_client = boto3.client('s3')
    print("Testing S3 GetObject permission")
    check_permissions(s3_client, 'get_object', Bucket='nonexistent-bucket', Key='nonexistent-key')

def test_cost_explorer_permissions():
    ce_client = boto3.client('ce')
    print("Testing Cost Explorer GetCostAndUsage permission")
    check_permissions(ce_client, 'get_cost_and_usage', TimePeriod={'Start': '2023-01-01', 'End': '2023-01-31'}, Granularity='MONTHLY', Metrics=['AmortizedCost'])

def test_budgets_permissions():
    budgets_client = boto3.client('budgets')
    print("Testing Budgets ViewBudget permission")
    check_permissions(budgets_client, 'describe_budget', AccountId='NonExistentAccountId', BudgetName='NonExistentBudget')

def test_cur_permissions():
    cur_client = boto3.client('cur')
    print("Testing CUR DescribeReportDefinitions permission")
    check_permissions(cur_client, 'describe_report_definitions')

def test_sagemaker_canvas_permissions():
    sagemaker_client = boto3.client('sagemaker')
    canvas_actions = [
        'create_canvas_app',
        'delete_canvas_app',
        'list_canvas_apps',
        'describe_canvas_app',
        'invoke_canvas_app',
        'start_canvas_app',
        'stop_canvas_app'
    ]
    for action in canvas_actions:
        print(f"Testing SageMaker Canvas {action} permission")
        check_permissions(sagemaker_client, action)

def test_permissions():
    print("Testing permissions for role")
    test_sagemaker_permissions()
    test_athena_permissions()
    test_quicksight_permissions()
    test_s3_permissions()
    test_cost_explorer_permissions()
    test_budgets_permissions()
    test_cur_permissions()
    test_sagemaker_canvas_permissions()


In [ ]:
role_name = 'studio-cur-user-profile'
role, policies, inline_policies = validate_iam_role(role_name)
if role:
    print(f"Role {role_name} details: {role}")
    print(f"Attached policies: {policies}")
    print(f"Inline policies: {inline_policies}")
    test_permissions()
else:
    print("Role validation failed.")